In [2]:
import requests
import pandas as pd
import openpyxl
from bs4 import BeautifulSoup
import time

https://www.infogreffe.fr/entreprise-societe/356000000-la-poste-750199B072480000.html

In [3]:
arx_book = openpyxl.load_workbook(r'C:\Users\Andy\Documents\3A\PROJET ARX\Projet Srapping_API - Partie 2.xlsx')
# book = openpyxl.load_workbook(r'C:\Outils_2\Python 3.74\Scripts\Projet PYNB\PROJET ARX\resultats.xlsx')
# writer= pd.ExcelWriter(r'C:\Outils_2\Python 3.74\Scripts\Projet PYNB\PROJET ARX\resultats.xlsx',engine='openpyxl')
# writer.book = book

In [4]:
sheet = arx_book.active

rows = sheet.max_row 
#la fonction ne renvoie pas la dernière ligne non-vide mais toutes les lignes d'où la condition !=None en bas
cols = sheet.max_column

SIRET = []
noms = []
pays = []

for row in sheet.iter_rows(min_row=2, min_col=2, max_row=rows, max_col=2) :
    for cell in row:
        if cell.value != None :
            noms.append(cell.value)
        
for row in sheet.iter_rows(min_row=2, min_col=3, max_row=rows, max_col=3) :
    for cell in row:
        if cell.value != None : 
            SIRET.append(cell.value)
        
for row in sheet.iter_rows(min_row=2, min_col=4, max_row=rows, max_col=4) :
    for cell in row:
        if cell.value != None : 
            pays.append(cell.value)
    
data = pd.DataFrame({'SIRET': SIRET, 'pays': pays, },
                      index = noms)

data

,SIRET,pays
AG2R La Mondiale,502 858 418 00028,France
Astera,570 501 551 00016,France
Groupe PRO BTP,394 164 966 00019,France
Crédit Mutuel,588 505 354 00013,France
Système U,304 602 956 00142,France
...,...,...
Greenergy,2809935,Royaume-Uni
Ineos,3851680,Royaume-Uni
Wittington Investments,366054,Royaume-Uni
Betfred Group,7717019,Royaume-Uni


In [105]:
SIRET_test = ['50285841800028','35600000000000','77557701800499', '64200799100069']
noms_test = ['AG2R La Mondiale']
prefixe = 'https://www.infogreffe.fr/infogreffe/ficheIdentite.do?siren='


def get_pages(U):
    pages = []
    for i in range(0,len(U)):
        SIREN = U[i]
        j = prefixe + SIREN[:-5] #enlever les 5 derniers pour avoir le SIREN à partir du SIRET
        pages.append(j)
    return pages

pages_test = get_pages(SIRET_test)
pages_test

['https://www.infogreffe.fr/infogreffe/ficheIdentite.do?siren=502858418',
 'https://www.infogreffe.fr/infogreffe/ficheIdentite.do?siren=356000000',
 'https://www.infogreffe.fr/infogreffe/ficheIdentite.do?siren=775577018',
 'https://www.infogreffe.fr/infogreffe/ficheIdentite.do?siren=642007991']

In [6]:
CA = []

for k in range(0,len(pages_test)):
    r = requests.get(pages_test[k]).text
    soup = BeautifulSoup(r,'html.parser')
    data = soup.find_all("div",{"class":"identTitreValeur", "id":"colChiffresCles"})
    for a in range(len(data)): 
        print(data[a].text+'hello')
            

 Derniers chiffres clés - SGAM AG2R LA MONDIALE     Clôture CA Résultat Effectif       hello


In [48]:
CA = []


for k in range(0,len(pages_test)):
    r = requests.get(pages_test[k]).text
    soup = BeautifulSoup(r,'html.parser')
    alldata1 = soup.find_all("td",{"class":"last"})
    for a in range(len(alldata1)):
        print(alldata1[a].text)

  Siège social  14-16 BOULEVARD MALESHERBES  75008 PARIS
  Siège social  9 RUE DU COLONEL PIERRE AVIA  75015 PARIS
  Siège social  1 RUE LOUIS LICHOU  29480 LE RELECQ-KERHUON
  Siège social  26 QUAI MARCEL BOYER  94200 IVRY-SUR-SEINE


In [19]:
len(alldata1)

1

In [107]:
def scrapping(pages_test):
    Comptes = []

    for k in range(0,len(pages_test)):
        r = requests.get(pages_test[k]).text
        soup = BeautifulSoup(r,'html.parser')
        alldata1 = soup.find_all("div",{"class":"identTitreValeur", "id":"colChiffresCles"})
        tableau = {}
        for a in range(len(alldata1)): 
            data = alldata1[a].find_all("div")
            l = []
            for b in range(len(data)):
                texte = data[b].text.replace('\xa0',' ')
                l.append(texte)
            Comptes.append(l)
    return Comptes

Comptes = scrapping(pages_test)
Comptes
        

[['    Clôture CA Résultat Effectif      '],
 ['    Clôture CA Résultat Effectif     31/12/2019  11 317 000 K€   -83 000 K€  182 183    31/12/2018  Comptes annuels déposés au Greffe. Commande possible.     31/12/2017  11 746 000 K€   659 000 K€  191 897    31/12/2016  11 882 000 K€   281 000 K€  197 398     '],
 ['    Clôture CA Résultat Effectif     31/12/2019  Comptes annuels déposés au Greffe. Commande possible.     31/12/2018  Comptes annuels déposés au Greffe. Commande possible.     31/12/2017  Comptes annuels déposés au Greffe. Commande possible.     31/12/2016  Comptes annuels déposés au Greffe. Commande possible.      '],
 ['    Clôture CA Résultat Effectif     31/12/2019  122 929 025 €    792    31/12/2018  118 504 537 €    781    31/12/2017  116 991 469 €    74    31/12/2016  108 483 575 €    702     ']]

len(alldata1) = 1, le premier résultat est le premier SIRET sauf que c'est vide, le deuxième est la Poste

In [163]:
def extraire_les_mots(texte, motif):
    maListe = []
    esp = texte.count(motif)
    deb = 0
    try:
        fin = texte.index(motif)
    except:
        return ['NA']
    

    for i in range(0, esp + 1):
        maListe.append(texte[deb:fin])
        texte=texte[fin+len(motif):]

        if texte.count(motif)!=0:
            fin = texte.index(motif)
        else:
            fin = len(texte)
            
    maListe[:] = [x for x in maListe if x != ''] #on enlève les ' ', car l'opération au-dessus laisse des "mots espaces"

    for i in range(len(maListe)) :
        if maListe[i][0] == " ":
            maListe[i] = maListe[i][1:] #on enlève les espaces devant les mots
        try :
            maListe[i] = time.strptime(maListe[i] , "%d/%m/%Y").tm_year #on convertit les dates en année
        except :
            None
            
    maListe = maListe[1:] #on enlève juste le premier élément
    
    return maListe


maListe = extraire_les_mots(Comptes[2][0][4:-5],"  ") #[4:-5] pour enlever les espaces avant et apres, du premier et seul élément
       
maListe

[2019,
 'Comptes annuels déposés au Greffe. Commande possible.',
 2018,
 'Comptes annuels déposés au Greffe. Commande possible.',
 2017,
 'Comptes annuels déposés au Greffe. Commande possible.',
 2016,
 'Comptes annuels déposés au Greffe. Commande possible. ']

In [160]:
import math

def trier(maListe):
    Cloture = []
    CA = []
    Résultat = []
    Effectif = []
    i = -1

    while i < len(maListe)-1:
        i = i + 1
        if type(maListe[i]) == int: #on vérifie que on a une année
            Cloture.append(maListe[i])
            continue

        if maListe[i] == 'Comptes annuels déposés au Greffe. Commande possible.' or maListe[i] == 'Comptes annuels déposés au Greffe. Commande possible. ':
            CA.append('NA')
            Résultat.append('NA')
            Effectif.append('NA')
            continue

        if maListe[i][-1] == '€' and maListe[i+1][-1] == '€': #on regarde si on a deux prix qui se suient
            CA.append(maListe[i])
            continue

        if maListe[i][-1] == '€' and maListe[i+1][-1] != '€': #car parfois le CA ou le Résultat est manquant, on fait l'hypothèse que s'il n'y qu'un prix alors c'est le CA
            if math.ceil((len(maListe)-4)/4) > 2:
                Résultat.append(maListe[i])
            else:
                CA.append(maListe[i])
            continue

        if maListe[i][-1] != '€' : #on a l'effectif, c'est le seul str qui ne finit pas par €
            Effectif.append(maListe[i])
    
    return Cloture, CA, Résultat, Effectif

Cloture, CA, Résultat, Effectif = trier(maListe)

print(Cloture)
print(CA)
print(Résultat)
print(Effectif)
        

[2019, 2018, 2017, 2016]
['NA', 'NA', 'NA', 'NA']
['NA', 'NA', 'NA', 'NA']
['NA', 'NA', 'NA', 'NA']


In [161]:
def creation_dataframe(Cloture, CA, Résultat, Effectif):
    
    taille = max(len(Cloture), len(CA), len(Résultat), len(Effectif))
    
    while len(Cloture) != taille:
        Cloture.append('NA')
    while len(CA) != taille:
        CA.append('NA')
    while len(Résultat) != taille:
        Résultat.append('NA')
    while len(Effectif) != taille:
        Effectif.append('NA')

    data = pd.DataFrame({'CA': CA, 'Résultat': Résultat, 'Effectif': Effectif, },
                          index = Cloture)

    return data
                 
data = creation_dataframe(Cloture, CA, Résultat, Effectif)
                 
data

    

,CA,Résultat,Effectif
2019,NA,NA,NA
2018,NA,NA,NA
2017,NA,NA,NA
2016,NA,NA,NA


In [164]:
for x in Comptes:
    maListe = extraire_les_mots(x[0][4:-5],"  ")
    Cloture, CA, Résultat, Effectif = trier(maListe)
    data = creation_dataframe(Cloture, CA, Résultat, Effectif)
                 
    display(data)


,CA,Résultat,Effectif
NA,NA,NA,NA


,CA,Résultat,Effectif
2019,11 317 000 K€,-83 000 K€,182 183
2018,NA,NA,NA
2017,11 746 000 K€,659 000 K€,191 897
2016,11 882 000 K€,281 000 K€,197 398


,CA,Résultat,Effectif
2019,NA,NA,NA
2018,NA,NA,NA
2017,NA,NA,NA
2016,NA,NA,NA


,CA,Résultat,Effectif
2019,122 929 025 €,NA,792
2018,118 504 537 €,NA,781
2017,116 991 469 €,NA,74
2016,108 483 575 €,NA,702


# Brouillon


In [44]:
#non adapté pour d'autres entreprises
Cloture = []
CA = []
Résultat = []
Effectif = []
i = 0

while i < len(maListe):
    if maListe[i+1] != 'Comptes annuels déposés au Greffe. Commande possible.':
        Cloture.append(maListe[i])
        CA.append(maListe[i+1])
        Résultat.append(maListe[i+2])
        Effectif.append(maListe[i+3])
        i = i + 3
    try :
        if maListe[i+1] == 'Comptes annuels déposés au Greffe. Commande possible.':
            Cloture.append(maListe[i])
            CA.append('NA')
            Résultat.append('NA')
            Effectif.append('NA')
            i = i + 1
    except :
        None
        
    i = i + 1
        
print(Cloture)
print(CA)
print(Résultat)
print(Effectif)

data = pd.DataFrame({'CA': CA, 'Résultat': Résultat, 'Effectif': Effectif, },
                      index = Cloture)

display(data)

[2019, 2018, 2017, 2016]
['11 317 000 K€', 'NA', '11 746 000 K€', '11 882 000 K€']
['-83 000 K€', 'NA', '659 000 K€', '281 000 K€']
['182 183', 'NA', '191 897', '197 398']


,CA,Résultat,Effectif
2019,11 317 000 K€,-83 000 K€,182 183
2018,NA,NA,NA
2017,11 746 000 K€,659 000 K€,191 897
2016,11 882 000 K€,281 000 K€,197 398


In [102]:
# ici c'est pour les petites tests
texte = "31/12/2019"
a = time.strptime(texte , "%d/%m/%Y")
print(a)
print(a.tm_year)

texte = "chien"
try :
    b = time.strptime(texte , "%d/%m/%Y")
except:
    print("An exception occurred")

time.struct_time(tm_year=2019, tm_mon=12, tm_mday=31, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=1, tm_yday=365, tm_isdst=-1)
2019
An exception occurred


In [143]:
type(int("u0")) == int

ValueError: invalid literal for int() with base 10: 'u0'